# Tutorial 3 - Memory

## Key Philosophy
- It would be important to learn from past experience and improve the agentic framework - memory is key to that
- You can add to the memory bank of your Agents pre-inference (by collecting from a pool of data prior to running the Agent), or during inference (add on in between running subtasks)

- In general, for the various Memory classes, you use:
    - `append(list_of_memories)`: append a list of memories to the memory bank
    - `retrieve(task: str)`: retrieves top_k memories based on task 
    
## Use Memory in Agents
- Agent class takes `memory_bank` as a parameter during initialisation of an `Agent`
- memory_bank: class Dict[Memory]. Stores multiple types of memory for use by the agent. Customise the Memory config within the Memory class.

# Setup Guide

## Step 1: Install AgentJo

In [1]:
# !pip install agentjo

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from agentjo import *
import os

from dotenv import load_dotenv
load_dotenv()

True

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Memory Class 1: Base Memory Class

- In-house Vector DB that can store anything in a Python list for retrieval to augment the Agent's prompt
- Retrieves top k memory items based on task 
- Inputs:
    - `memory`: List. Default: Empty List. The list containing the memory items
    - `top_k`: Int. Default: 5. The number of memory list items to retrieve
    - `mapper`: Function. Maps the memory item to another form for comparison by ranker or LLM. Default: `lambda x: x`
        - Example mapping: `lambda x: x.fn_description` (If x is a Class and the string you want to compare for similarity is the fn_description attribute of that class)
    - `approach`: str. Either `retrieve_by_ranker` or `retrieve_by_llm` to retrieve memory items.
        - Ranker is faster and cheaper as it compares via embeddings, but are inferior to LLM-based methods for contextual information
    - `retrieve_fn`: Default: None. Takes in task and outputs top_k similar memories in a list
    - `ranker`: `Ranker`. The Ranker which defines a similarity score between a query and a key. Default: OpenAI `text-embedding-3-small` model. 
        - Can be replaced with a function which returns similarity score from 0 to 1 when given a query and key
        
## In-built Memory in AgentJo: Function Memory
- Default: `memory_bank = {'Function': Memory(top_k = 5, mapper = lambda x: x.fn_description, approach = 'retrieve_by_ranker'), llm = self.llm}`
- Does RAG over Task -> Function mapping

## Use Case 1: Filtering Functions by Task
- AgentJo chooses `top k` (default k: 5) functions according to similarity to subtask
- In addition to `top k` functions, we will also give agent all the compulsory functions
    - `is_compulsory` variable of Function set to `True` means that we will always have it as one of the functions for planning and bypass Function RAG
    
## Example Use Case
- Helps to reduce number of functions present in LLM context for more accurate generation
```python
output = my_agent.run('Calculate 2**10 * (5 + 1) / 10')
```
`Original Function List: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, divide_numbers, power_of, GCD_of_two_numbers, modulo_of_numbers, absolute_difference, generate_poem_with_numbers, List_related_words, generate_quote`

`Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, power_of, modulo_of_numbers`


In [5]:
from typing import List
import math
def sum_numbers(num_list: List[float]) -> float:
    '''Adds all numbers in num_list'''
    return sum(x for x in num_list)

def subtract_numbers(num1: float, num2: float) -> float:
    '''Subtracts num1 from num2'''
    return num1 - num2

def multiply_numbers(num1: float, num2: float) -> float:
    '''Multiplies num1 by num2'''
    return num1 * num2

def divide_numbers(num1: float, num2: float) -> float:
    '''Divides num1 by num2'''
    if num2 == 0:
        return -1
    return num1/num2

def power_operation(num1: float, num2: float) -> float:
    '''Returns num1 to the power of num2 (num1**num2)'''
    return math.pow(num1, num2)

def greatest_common_divisor(num1: int, num2: int) -> int:
    '''Returns greatest common divisor of num1 and num2'''
    return math.gcd(num1, num2)

def modulo(num1: int, num2: int) -> int:
    '''Returns modulo of num1 over num2'''
    return num1%num2

def absolute_difference(num1: int, num2: int) -> int:
    '''Returns absolute difference between num1 and num2'''
    return math.abs(num1-num2)

# Put this to make sum_numbers always appear for any task and bypass Function RAG
sum_numbers = Function(external_fn = sum_numbers, is_compulsory = True)

# This is for Internal Functions
generate_poem_with_numbers = Function("Generates a poem containing <num1: float> and <num2: float>", output_format = {"Poem": "Poem"}, fn_name = 'generate_poem_with_numbers', llm = llm)
list_related_words = Function("Lists out <num: int> words related to <word: str>", output_format = {"List of words": "List of words, type: list"}, fn_name = 'list_related_words', llm = llm)
generate_quote = Function("Generates a quote about <topic: str>", output_format = {"Quote": "Quote"}, fn_name = 'generate_quote', llm = llm)

In [6]:
my_agent = Agent('Generalist Agent', 
'''Does everything''',
                default_to_llm = False,
                llm = llm).assign_functions([sum_numbers, subtract_numbers, multiply_numbers, 
            divide_numbers, power_operation, greatest_common_divisor, modulo, absolute_difference, 
            generate_poem_with_numbers, list_related_words, generate_quote])

In [7]:
# see the auto-generated names of your functions :)
my_agent.print_functions()

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: sum_numbers
Description: Adds all numbers in <num_list: list[float]>
Input: ['num_list']
Output: {'output_1': 'float'}

Name: subtract_numbers
Description: Subtracts <num1: float> from <num2: float>
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: multiply_numbers
Description: Multiplies <num1: float> by <num2: float>
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: divide_numbers
Description: Divides <num1: float> by <num2: float>
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: power_operation
Description: Returns <num1: float> to the power of <num2: float> (<num1: float>**<num2: float>)
Input: ['num1', 'num2']
Output: {'output_1': 'float'}

Name: greatest_common_divisor
Description: Returns greatest common divisor of <num1: int> and <num2: int>
Input: ['num1', 'num2']
Output: {'output_1': 'int'}

Name: modulo
Description: Returns modulo of <num1: int> over 

In [8]:
# Configure your top_k for function filtering here, default is 5
my_agent.memory_bank['Function'].top_k = 5

In [9]:
# visualise how the Functions are chosen based on task - here you see subtract_numbers appearing at the front
# this does not include the compulsory functions
[f.fn_name for f in my_agent.memory_bank['Function'].retrieve('Evaluate 3 - 1')]

['modulo',
 'subtract_numbers',
 'multiply_numbers',
 'divide_numbers',
 'absolute_difference']

In [10]:
my_agent.reset()
my_agent.run('Evaluate 2+3')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The task is to evaluate the expression 2 + 3. No subtasks have been completed yet.

Thoughts: To complete the assigned task, I need to perform the addition of the numbers 2 and 3.

Subtask identified: Use the equipped function to add the numbers 2 and 3.

Calling function sum_numbers with parameters {'num_list': [2.0, 3.0]}
> {'output_1': 5.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The sum of 2 and 3 has been calculated, resulting in 5.0.

Thoughts: Since the Assigned Task is to evaluate 2 + 3 and the result has already been computed, the next step is to finalize the task and present the output to the user.

Subtask identified: End Task
Task completed successfully!



[{'output_1': 5.0}]

In [11]:
my_agent.reset()
output = my_agent.run('Evaluate 2**10 * (3+5) / 10')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The Assigned Task requires evaluating the expression 2**10 * (3+5) / 10. 
No subtasks have been completed yet.

Thoughts: To complete the Assigned Task, I need to break down the expression into manageable parts. 
First, I can calculate the exponentiation (2**10), then evaluate the addition (3+5), 
and finally perform the multiplication and division in sequence.

Subtask identified: Calculate the value of 2**10 and store the result for further operations.

Calling function power_operation with parameters {'num1': 2.0, 'num2': 10.0}
> {'output_1': 1024.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The power operation has been completed, resulting in 1024.0 from 2 raised to the power of 10.

Thoughts: To complete the remainder of the assigned task, I need to ev

In [12]:
my_agent.thoughts

[{'Observation': 'The Assigned Task requires evaluating the expression 2**10 * (3+5) / 10. \nNo subtasks have been completed yet.\n',
  'Thoughts': 'To complete the Assigned Task, I need to break down the expression into manageable parts. \nFirst, I can calculate the exponentiation (2**10), then evaluate the addition (3+5), \nand finally perform the multiplication and division in sequence.\n',
  'Current Subtask': 'Calculate the value of 2**10 and store the result for further operations.\n',
  'Equipped Function Name': 'power_operation',
  'Equipped Function Inputs': {'num1': 2.0, 'num2': 10.0}},
 {'Observation': 'The power operation has been completed, resulting in 1024.0 from 2 raised to the power of 10.\n',
  'Thoughts': 'To complete the remainder of the assigned task, I need to evaluate the expression (3 + 5) and then multiply the result by 1024.0. After that, I will divide the product by 10 to get the final result.\n',
  'Current Subtask': 'Calculate (3 + 5) and then multiply the 

In [13]:
my_agent.reply_user()

'To evaluate the expression 2**10 * (3+5) / 10, we can break it down using the subtasks that have already been completed.\n\n1. **Calculate 2**10**: \n   - This is equivalent to `power_operation(num1=2.0, num2=10.0)`, which has been completed with an output of **1024.0**.\n\n2. **Calculate (3+5)**: \n   - This is equivalent to `sum_numbers(num_list=[3.0, 5.0])`, which has been completed with an output of **8.0**.\n\n3. **Multiply the results of the first two steps**: \n   - We need to calculate **1024.0 * 8.0**. This is equivalent to `multiply_numbers(num1=1024.0, num2=8.0)`, which has been completed with an output of **8192.0**.\n\n4. **Divide the result by 10**: \n   - Finally, we need to calculate **8192.0 / 10.0**. This is equivalent to `divide_numbers(num1=8192.0, num2=10.0)`, which has been completed with an output of **819.2**.\n\nTherefore, the final result of the expression 2**10 * (3+5) / 10 is **819.2**.\n'

In [14]:
my_agent.status()

Agent Name: Generalist Agent
Agent Description: Does everything
Available Functions: ['end_task', 'sum_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power_operation', 'greatest_common_divisor', 'modulo', 'absolute_difference', 'generate_poem_with_numbers', 'list_related_words', 'generate_quote']
Shared Variables: ['agent']
Task: Evaluate 2**10 * (3+5) / 10
Subtasks Completed:
Subtask: power_operation(num1=2.0, num2=10.0)
{'output_1': 1024.0}

Subtask: sum_numbers(num_list=[3.0, 5.0])
{'output_1': 8.0}

Subtask: multiply_numbers(num1=1024.0, num2=8.0)
{'output_1': 8192.0}

Subtask: divide_numbers(num1=8192.0, num2=10.0)
{'output_1': 819.2}

Subtask: Replying User Query: Evaluate 2**10 * (3+5) / 10
To evaluate the expression 2**10 * (3+5) / 10, we can break it down using the subtasks that have already been completed.

1. **Calculate 2**10**: 
   - This is equivalent to `power_operation(num1=2.0, num2=10.0)`, which has been completed with an output of **1024.0**.

2

## Use Case 2: Adding more context based on task
- You can add task-dependent context to the memory_bank so that `top k` will be added to prompt based on task

### Using `memory_bank` for more context
- Here, we have a mapping of nonsense words to numbers
- Based on the subtask, we will augment the system prompt with relevant mappings to aid planning

In [15]:
# first append more context to the agent
my_agent = Agent('Poem Creator', 'Create a poem according to a name', llm = llm)
my_agent.memory_bank['Task Instructions'] = Memory(['For John, generate a poem about the sun', 
'For Mary, generate a poem about the wind', 
'For Peter, generate a poem about the sea'], # some task-specific instructions
            top_k = 1,  # choose top 1
            mapper = lambda x: x) # we compare with the task using only the first word, e.g. Azo, Boneti, Andkh

In [16]:
# If name is present in memory, can use this information (e.g. John -> sun)
my_agent.reset()
output = my_agent.run('John')

Observation: A poem about John has not yet been created, but the task is to generate a poem about the sun for him.

Thoughts: To complete the remainder of the Assigned Task, I need to create a poem that captures the essence of the sun, focusing on its beauty, warmth, and significance.

Subtask identified: Generate the first stanza of a poem about the sun for John.

Getting LLM to perform the following task: Generate the first stanza of a poem about the sun for John.

> In the morning's gentle embrace, the sun does rise,  
A golden orb that paints the skies,  
With rays of warmth, it greets the day,  
Chasing the shadows, lighting the way.  


Observation: A stanza about the sun for John has been successfully generated, capturing its beauty and warmth.

Thoughts: To complete the remainder of the poem for John, I need to generate additional stanzas that continue the theme of the sun, perhaps exploring its impact on nature or emotions.

Subtask identified: Generate the second stanza of a 

In [17]:
# If the name is not in the memory, it will be interpreted according to the actual input
my_agent.reset()
output = my_agent.run('Isaac Newton')

Observation: No subtasks have been completed for the task of creating a poem about Isaac Newton.

Thoughts: To complete the remainder of the task, I need to generate a poem that reflects the life and contributions of Isaac Newton, focusing on his discoveries in physics and mathematics.

Subtask identified: Generate a poem about Isaac Newton using the equipped function.

Getting LLM to perform the following task: Generate a poem about Isaac Newton using the equipped function.

> In the realm of thought, where shadows play,
Isaac Newton, a mind that lit the way.
Beneath the apple tree, a moment profound,
The laws of motion and gravity he found.

With a gaze to the heavens, he pondered the night,
The orbs in their dance, a celestial sight.
From calculus' birth to the prism's bright hue,
He unveiled the secrets, the universe anew.

A force that pulls, a force that guides,
In every falling leaf, his wisdom abides.
The world spins in harmony, a cosmic ballet,
Thanks to Newton's brilliance, w

## Use Case 3: Add memory directly using pdf, docx, csv, xls files
Adding memory elements one by one can be cumbersome, AgentJo memory can take filepath as input and it will split the text content inside the file path either using default splitter or user provided splitter.

- We currently support `pdf`, `docx`, `csv`, `xls` files
- You can also input own `text_splitter` function which takes in text and returns a list of splitted text by that function. Default: LangChain's RecursiveCharacterTextSplitter
- We recommend the async method for faster accessing of memory

Example:

`memory = Memory(top_k = 5)`

`memory.add_file(file_path)`


In [18]:
# ## Run the sync version of add_file (Takes very long)

# import time

# start_time = time.time()
# memory = Memory(top_k = 1)
# memory.add_file(filepath="./react.pdf")
# memory.retrieve('What is react')

# end_time = time.time()
# print("Time taken to run the code:", end_time - start_time, "seconds")

In [19]:
# Async version of memory (Faster)
import time

start_time = time.time()
async_memory = AsyncMemory(top_k = 3)
async_memory.add_file(filepath="./react.pdf")
await async_memory.retrieve('What is react')

end_time = time.time()
print("Time taken to run the code:", end_time - start_time, "seconds")

Time taken to run the code: 2.986531972885132 seconds


In [20]:
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [21]:
# Equip this memory to the Agent and use it to answer a question
agent = AsyncAgent('Content Answerer', 'Replies to questions factually based on given memory', llm = llm_async)

In [22]:
agent.memory_bank['Document'] = async_memory

In [23]:
await async_memory.retrieve('What is ReAct?')

['erage success rate of 71%, signiﬁcantly outperforming\nthe best Act (45%) and BUTLER (37%) trials. In fact, even the worse ReAct trial (48%) beats\nthe best trial of both methods. Moreover, the advantage of ReAct overAct is consistent across\nsix controlled trials, with relative performance gain ranging from 33% to 90% and averaging 62%.\nQualitatively, we saw that, without any thoughts at all, Act fails to correctly decompose goals\ninto smaller subgoals, or loses track of the current state of the environment. Example trajectories\ncomparing ReAct andAct can be found in Appendix D.2.1 and Appendix D.2.2.\nOn Webshop, one-shot Act prompting already performs on par with IL and IL+RL methods. With\nadditional sparse reasoning, ReAct achieves signiﬁcantly better performance, with an absolute 10%\nimprovement over the previous best success rate. By checking examples, we ﬁnd that ReAct is more\nlikely to identify instruction-relevant products and options by reasoning to bridge the gap bet

In [24]:
await agent.run('What is ReAct?')

Observation: ReAct is a method that integrates reasoning and action in a flexible manner, allowing for dynamic decision-making across various tasks. It has shown a significant performance advantage over traditional methods like Act, particularly in tasks such as question answering and web navigation. The empirical evaluations indicate that ReAct can effectively generate reasoning traces and actions, leading to improved outcomes in diverse benchmarks.

Thoughts: To complete the remainder of the Assigned Task, I need to summarize the key features and benefits of ReAct, emphasizing its performance, flexibility, and human alignment. This will provide a comprehensive understanding of what ReAct is and how it functions in various contexts.

Subtask identified: Summarize the key features and benefits of ReAct, focusing on its performance advantages, flexibility in handling diverse tasks, and its human-aligned decision-making process.

Getting LLM to perform the following task: Summarize the k

["ReAct is a versatile framework designed to enhance performance across a variety of tasks, showcasing several key features and benefits:\n\n- **Performance Advantages**: ReAct demonstrates a significant improvement in success rates compared to traditional methods. In benchmarks like HotpotQA and Fever, it effectively mitigates issues of hallucination and error propagation by utilizing a Wikipedia API for real-time information retrieval. This results in more accurate and interpretable task-solving trajectories, with ReAct achieving an average success rate of 71%, outperforming the best alternatives by a notable margin.\n\n- **Flexibility in Handling Diverse Tasks**: The framework is built to accommodate a wide range of action spaces and reasoning requirements. Its flexible thought-action occurrence format allows it to adapt to various tasks, including question answering, fact verification, text games, and web navigation. This adaptability is crucial for generalization to new task insta